# The Battle of Neighborhoods

### 1 - Description

In this project we will compare neighborhoods of different cities based on the venues each neighborhood contains. The question we will be answering is: Are neighborhoods in one cities more similar to each other than they are similar to neighborhoods in other cities? Would city centers cluster together?

In other part, we will be trying to provide an advice for an entrepreneur regarding where they should open a restaurent. We will be looking at the density of restaurants in each neighborhood. We assume that a restauant will be more profitable in an area with a low density of restaurents. 

### 2 - Data requirements

The data we will require is: 
- A List of the neighborhoods in each city with 
- Venues data for each location will be retrived using the Foursquare API

The required libraries: 
- Pandas 
- Numpy
- Sklearn and more specifically the kmeans method
- Geocoder to obtain the location of each neighborhood
- BeautifulSoup to scrap data from wikipedia for the city of Toronto and maybe other cities